In [13]:
import pandas as pd
import re

def dividir_ubicacion(ubicacion_str):
    """
    Divide la cadena de ubicación en departamento, provincia y distrito
    Ejemplo: "Amazonas, Chachapoyas, distrito: Chachapoyas" →
    {"departamento": "Amazonas", "provincia": "Chachapoyas", "distrito": "Chachapoyas"}
    """
    if pd.isna(ubicacion_str) or not isinstance(ubicacion_str, str):
        return {"departamento": "", "provincia": "", "distrito": ""}
    
    try:
        # Dividir por comas
        partes = [parte.strip() for parte in ubicacion_str.split(',')]
        
        departamento = partes[0] if len(partes) > 0 else ""
        provincia = partes[1] if len(partes) > 1 else ""
        
        # Extraer distrito (puede ser "distrito: Nombre" o solo el nombre)
        distrito = ""
        if len(partes) > 2:
            distrito_str = partes[2]
            if 'distrito:' in distrito_str.lower():
                distrito_match = re.search(r'distrito:\s*(.+)', distrito_str, re.IGNORECASE)
                distrito = distrito_match.group(1).strip() if distrito_match else distrito_str
            else:
                distrito = distrito_str
        
        return {
            "departamento": departamento,
            "provincia": provincia,
            "distrito": distrito
        }
    except:
        return {"departamento": "", "provincia": "", "distrito": ""}

def procesar_censo_completo(df, año_censo):
    """
    Procesa el censo y divide la ubicación en departamento, provincia y distrito
    """
    datos = []
    ubigeo_actual = ""
    ubicacion_actual = ""
    
    for i in range(len(df)):
        fila = df.iloc[i].tolist()
        
        # Buscar línea con AREA # (columna B - índice 1)
        if len(fila) > 1 and pd.notna(fila[1]) and 'AREA #' in str(fila[1]):
            # Extraer ubigeo
            match = re.search(r'AREA #\s*(\d{6})', str(fila[1]))
            if match:
                ubigeo_actual = match.group(1)
            
            # Extraer ubicación completa (columna C - índice 2)
            if len(fila) > 2 and pd.notna(fila[2]):
                ubicacion_actual = str(fila[2]).strip()
                
                # Dividir la ubicación
                ubicacion_dividida = dividir_ubicacion(ubicacion_actual)
                print(f"📌 {ubigeo_actual} - {ubicacion_dividida}")
            continue
        
        # Buscar datos de población (Urbano censal, Rural censal, Total)
        if (len(fila) > 1 and pd.notna(fila[1]) and 
            any(x in str(fila[1]) for x in ['Urbano censal', 'Rural censal', 'Total'])):
            
            try:
                tipo_area = str(fila[1]).strip()
                # Población en columna C (índice 2)
                if len(fila) > 2 and pd.notna(fila[2]):
                    # Limpiar número
                    num_str = str(fila[2]).replace(' ', '').replace(',', '')
                    poblacion = int(float(num_str))
                    
                    # Dividir la ubicación actual
                    ubicacion_dividida = dividir_ubicacion(ubicacion_actual)
                    
                    datos.append({
                        'ubigeo': ubigeo_actual,
                        'departamento': ubicacion_dividida['departamento'],
                        'provincia': ubicacion_dividida['provincia'],
                        'distrito': ubicacion_dividida['distrito'],
                        'tipo_area': tipo_area,
                        'poblacion': poblacion,
                        'anio': año_censo
                    })
                    print(f"   ➡️ {tipo_area}: {poblacion}")
            except Exception as e:
                print(f"Error procesando fila {i}: {e}")
                continue
    
    return pd.DataFrame(datos)

# Función alternativa para manejar diferentes formatos de ubicación
def dividir_ubicacion_avanzada(ubicacion_str):
    """
    Versión más robusta para dividir ubicación
    """
    if pd.isna(ubicacion_str) or not isinstance(ubicacion_str, str):
        return {"departamento": "", "provincia": "", "distrito": "", "ubicacion_original": ubicacion_str}
    
    try:
        # Patrones comunes
        patrones = [
            r'(.*?),\s*(.*?),\s*distrito:\s*(.+)',  # "Depto, Prov, distrito: Dist"
            r'(.*?),\s*(.*?),\s*(.+)',              # "Depto, Prov, Dist"
            r'(.*?),\s*(.+)',                       # "Depto, Dist" 
            r'(.*?)'                                # Solo Depto
        ]
        
        for patron in patrones:
            match = re.match(patron, ubicacion_str, re.IGNORECASE)
            if match:
                grupos = match.groups()
                departamento = grupos[0].strip()
                provincia = grupos[1].strip() if len(grupos) > 1 else ""
                distrito = grupos[2].strip() if len(grupos) > 2 else grupos[1].strip() if len(grupos) > 1 else ""
                
                # Limpiar "distrito:" si está presente
                if 'distrito:' in distrito.lower():
                    distrito = re.sub(r'distrito:\s*', '', distrito, flags=re.IGNORECASE).strip()
                
                return {
                    "departamento": departamento,
                    "provincia": provincia,
                    "distrito": distrito
                }
        
        # Si no coincide con ningún patrón, devolver todo como departamento
        return {
            "departamento": ubicacion_str,
            "provincia": "",
            "distrito": ""
        }
        
    except:
        return {"departamento": "", "provincia": "", "distrito": ""}

# EJECUCIÓN COMPLETA
print("🚀 INICIANDO PROCESAMIENTO COMPLETO")
ruta = r"D:\Mateo\ICSI\Proyecto\Data\Raw\Poblacion\Data\Inputs\Censo\censo_pob_area_2017.xlsx"
df = pd.read_excel(ruta, header=None)

# Procesar con división de ubicación
resultado = procesar_censo_completo(df, 2017)

if not resultado.empty:
    print(f"\n✅ PROCESAMIENTO COMPLETO EXITOSO!")
    print(f"📊 Total de registros: {len(resultado)}")
    
    # Mostrar estadísticas
    print(f"\n📈 ESTADÍSTICAS:")
    print(f"Departamentos únicos: {resultado['departamento'].nunique()}")
    print(f"Provincias únicas: {resultado['provincia'].nunique()}")
    print(f"Distritos únicos: {resultado['distrito'].nunique()}")
    print(f"Tipos de área: {resultado['tipo_area'].unique()}")
    
    print(f"\n👀 PRIMEROS 10 REGISTROS:")
    print(resultado[['ubigeo', 'departamento', 'provincia', 'distrito', 'tipo_area', 'poblacion']].head(10))
    
    # Guardar resultados
    ruta_destino = r"D:\Mateo\ICSI\Proyecto\Data\Raw\Poblacion\Data\Processing\pob_censo_area_2017.csv"
    resultado.to_csv(ruta_destino, index=False, encoding='latin1')
    print(f"\n💾 Archivo guardado como 'censo_2017_completo.csv'")
    
    # Opcional: guardar versión simplificada
    resultado[['ubigeo', 'departamento', 'provincia', 'distrito', 'tipo_area', 'poblacion', 'anio']].to_csv(
        'censo_2017_simple.csv', index=False, encoding='latin1'
    )
    print("💾 Versión simplificada guardada como 'censo_2017_simple.csv'")
    
else:
    print("❌ No se encontraron datos")
    print("🔍 Mostrando estructura del archivo:")
    for i in range(15):
        fila = df.iloc[i].tolist()
        valores = [str(x) for x in fila if pd.notna(x)]
        if valores:
            print(f"Fila {i}: {valores}")

🚀 INICIANDO PROCESAMIENTO COMPLETO
📌 010101 - {'departamento': 'Amazonas', 'provincia': 'Chachapoyas', 'distrito': 'Chachapoyas'}
   ➡️ Urbano censal: 35233
   ➡️ Rural censal: 634
   ➡️ Total: 35867
📌 010102 - {'departamento': 'Amazonas', 'provincia': 'Chachapoyas', 'distrito': 'Asunción'}
   ➡️ Urbano censal: 155
   ➡️ Rural censal: 127
   ➡️ Total: 283
📌 010103 - {'departamento': 'Amazonas', 'provincia': 'Chachapoyas', 'distrito': 'Balsas'}
   ➡️ Urbano censal: 371
   ➡️ Rural censal: 861
   ➡️ Total: 1232
📌 010104 - {'departamento': 'Amazonas', 'provincia': 'Chachapoyas', 'distrito': 'Cheto'}
   ➡️ Urbano censal: 473
   ➡️ Rural censal: 208
   ➡️ Total: 681
📌 010105 - {'departamento': 'Amazonas', 'provincia': 'Chachapoyas', 'distrito': 'Chiliquin'}
   ➡️ Urbano censal: 200
   ➡️ Rural censal: 427
   ➡️ Total: 628
📌 010106 - {'departamento': 'Amazonas', 'provincia': 'Chachapoyas', 'distrito': 'Chuquibamba'}
   ➡️ Urbano censal: 570
   ➡️ Rural censal: 1408
   ➡️ Total: 1978
📌 010107